In [2]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader
from src.inception import inception_v3

train_transforms = transforms.Compose([
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2470, 0.2435, 0.2616])
])

test_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.4914, 0.4822, 0.4465], std=[0.2470, 0.2435, 0.2616])
])

train_dataset = CIFAR10(root="../../data", train=True, transform=train_transforms, download=True)
test_dataset = CIFAR10(root="../../data", train=False, transform=test_transforms, download=True)

train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True, num_workers=4)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False, num_workers=4)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = inception_v3(pretrained=False).to(device)

criterion = nn.CrossEntropyLoss()

def evaluate_model(model, test_loader, device):
    model.eval()
    correct, total = 0, 0

    with torch.no_grad():
        for images, labels in test_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

    accuracy = 100 * correct / total
    return accuracy  

def train_model(model, train_loader, criterion, optimizer, device, num_epochs=10):
    model.train()
    for epoch in range(num_epochs):
        running_loss, correct, total = 0.0, 0, 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            running_loss += loss.item()
            _, predicted = outputs.max(1)
            total += labels.size(0)
            correct += predicted.eq(labels).sum().item()

        accuracy = 100 * correct / total
        print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader):.4f}, Accuracy: {accuracy:.2f}%\n")
        
        if accuracy > 90:
            print("Testing Accuracy on Test Set...")
            test_accuracy = evaluate_model(model, test_loader, device)
            if test_accuracy > 91:
                print(f"Model accuracy reaches {test_accuracy}% at epoch {epoch}")
                print("Training Finished!")
                break
            else:
                print("Model haven't arrived 92% accuracy. Training continue!\n")
                model.train()

optimizer = torch.optim.Adam(model.parameters(), lr=1e-3, weight_decay=1e-4)
optimizer_1 = torch.optim.Adam(model.parameters(), lr=1e-4, weight_decay=5e-5)
train_model(model, train_loader, criterion, optimizer, device, num_epochs=20)
train_model(model, train_loader, criterion, optimizer_1, device, num_epochs=10)


Files already downloaded and verified
Files already downloaded and verified
Epoch 1/20, Loss: 1.5509, Accuracy: 41.44%

Epoch 2/20, Loss: 1.0609, Accuracy: 62.12%

Epoch 3/20, Loss: 0.8466, Accuracy: 70.31%

Epoch 4/20, Loss: 0.7085, Accuracy: 75.58%

Epoch 5/20, Loss: 0.6269, Accuracy: 78.74%

Epoch 6/20, Loss: 0.5658, Accuracy: 80.81%

Epoch 7/20, Loss: 0.5129, Accuracy: 82.63%

Epoch 8/20, Loss: 0.4676, Accuracy: 84.16%

Epoch 9/20, Loss: 0.4419, Accuracy: 85.23%

Epoch 10/20, Loss: 0.4073, Accuracy: 86.14%

Epoch 11/20, Loss: 0.3852, Accuracy: 87.19%

Epoch 12/20, Loss: 0.3658, Accuracy: 87.50%

Epoch 13/20, Loss: 0.3410, Accuracy: 88.54%

Epoch 14/20, Loss: 0.3244, Accuracy: 89.09%

Epoch 15/20, Loss: 0.3098, Accuracy: 89.59%

Epoch 16/20, Loss: 0.2922, Accuracy: 89.93%

Epoch 17/20, Loss: 0.2788, Accuracy: 90.53%

Testing Accuracy on Test Set...
Model haven't arrived 92% accuracy. Training continue!

Epoch 18/20, Loss: 0.2631, Accuracy: 91.05%

Testing Accuracy on Test Set...
Mod

In [3]:
test_accuracy = evaluate_model(model, test_loader, device)
print(f"Current Accuracy on Test Set: {test_accuracy}%")

Current Accuracy on Test Set: 92.39%


In [4]:
import os
save_dir = "C:\\Users\\jiayang\\ipynb\\trainedModel" # your save save path
if not os.path.exists(save_dir):
    os.makedirs(save_dir)
    
model_path = os.path.join(save_dir, "Inception_CIFAR10.pth")
torch.save(model.state_dict(), model_path)
print(f"Model saved to {model_path}")

Model saved to C:\Users\jiayang\ipynb\trainedModel\Inception_CIFAR10.pth
